In [20]:
from bs4 import BeautifulSoup 
from selenium import webdriver
# The following packages will also be used in this tutorial
import pandas as pd # All database operations
import numpy as np  # Numerical operations
import time         # Tracking time
import requests     # HTTP requests
import re           # String manipulation
from sklearn.feature_extraction.text import CountVectorizer # BagOfWords (cleaning)
from joblib import Parallel, delayed # Parallellization of tasks

In [122]:
def getPage(url):
	''' returns a soup object that contains all the information 
	of a certain webpage'''
	result = requests.get(url)
	content = result.content
	return BeautifulSoup(content, features = "lxml")

url_page ="https://www.airbnb.fr/s/Paris/homes?adults=1&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&checkin=2020-11-25&checkout=2020-11-28&source=structured_search_input_header&search_type=filter_change"
page=getPage(url_page)

In [123]:
def getRoomClasses(soupPage):
	''' This function returns all the listings that can 
	be found on the page in a list.'''
	rooms = soupPage.findAll("div", {"class": "_8ssblpx"})
	result = []
	for room in rooms:
		result.append(room)
	return result
listing=getRoomClasses(page)

In [124]:
##commentaires
def getNbCommentaire(listing):
	''' Returns the guest information'''
	try:
		return listing.find("span", {"class":"_a7a5sx"}).text 
	except:
		return "pas de commentaire"

In [125]:
##evaluation
def getEvaluation(listing):
	''' Returns the guest information'''
	try:
		return listing.find("span", {"class":"_10fy1f8"}).text 
	except:
		return "pas d'évaluation"


In [126]:
## lien du logement
def getListingLink(listing):
	''' Returns the guest information'''
	try:
		return "http://airbnb.com" + listing.find("a")["href"]
	except:
		return "pas de lien"

In [127]:
## titre du logement
def getListingTitle(listing):
	''' Returns the guest information'''
	try:
		return listing.find("meta")["content"]
	except:
		return "pas de titre du logement"

In [128]:
##description du logement
def getTopRow(listing):
	''' Returns the guest information'''
	try:
		return listing.find("div", {"class": "_167qordg"}).text
	except:
		return "pas de description du logement"

In [129]:
##information sur le nombre de chambre/douche
def getRoomInfo(listing):
	''' Returns the guest information'''
	try:
		return listing.find("div", {"class":"_kqh46o"}).text
	except:
		return "pas d'infos sur chambre/douche"

In [130]:
##prix
def getPrix(listing):
	''' Returns the guest information'''
	try:
		return listing.find("div", {"class":"_1fwiw8gv"}).text
	except:
		return "pas d'infos sur le prix"

In [131]:

getNbCommentaire(listing[5])
getPrix(listing[0])
getEvaluation(listing[5])
getListingTitle(listing[6])

'★Jolie chambre Double proche de Bastille - null - Paris'

In [140]:
## extraction des infos de tous les logements d'une page

def extractinfo(page):
    df = pd.DataFrame(columns = ['title', 'link',"nbComments","prix","topRow","evaluation","roomInfo"]) ##,"evaluation", ,'nbComments'
    new=[]
    listing=getRoomClasses(page)
    for i in range(1, len(listing)):
        new.append(getListingTitle(listing[i]))
        new.append(getListingLink(listing[i]))
        new.append(getNbCommentaire(listing[i]))
        new.append(getPrix(listing[i]))
        new.append(getTopRow(listing[i]))
        new.append(getEvaluation(listing[i]))
        new.append(getRoomInfo(listing[i]))
        df.loc[i]=new
        new=[]
    return df

In [141]:
extractinfo(page)

,title,link,nbComments,prix,topRow,evaluation,roomInfo
1,Joli studio à deux pas du Canal de l'Ourcq - n...,http://airbnb.com/rooms/14724032?adults=1&chec...,(111),Prix :48€ / nuit,Logement entier à La Villette,4.52,2 voyageurs · Studio · 1 lit · 1 salle de bain
2,Amazing studio in Pigalle near Montmartre - nu...,http://airbnb.com/rooms/45678910?adults=1&chec...,(4),Prix initial :71€Prix réduit :50€ / nuit,Logement entier à 9e arrondissement,4.25,2 voyageurs · Studio · 0 lit · 1 salle de bain
3,French Theory - La Chambre d'Etude - null - P...,http://airbnb.com/rooms/39410968?adults=1&chec...,(116),Prix initial :61€Prix réduit :56€ / nuit,Chambre dans un boutique-hôtel à Quartier Latin,4.80,1 voyageur · 1 chambre · 1 lit · 1 salle de ba...
4,Sunny & Cosy Apartment — Paris Centre - null -...,http://airbnb.com/rooms/45284087?adults=1&chec...,(5),Prix initial :89€Prix réduit :78€ / nuit,Logement entier à 9e arrondissement,4.60,2 voyageurs · 1 chambre · 1 lit · 1 salle de bain
5,Petit 2 pièces avec terrasse - null - Paris,http://airbnb.com/rooms/21575578?adults=1&chec...,(213),Prix initial :67€Prix réduit :49€ / nuit,Suite entière à 10e arrondissement,4.45,"4 voyageurs · 1 chambre · 0 lit · 1,5 salle de..."
6,★Jolie chambre Double proche de Bastille - nul...,http://airbnb.com/rooms/45555163?adults=1&chec...,(3),Prix :43€ / nuit,Chambre d'hôtel à 11e arrondissement,3.67,2 voyageurs · 1 chambre · 1 lit · 1 salle de b...
7,Charmante Studette Saint-Augustin - null - Paris,http://airbnb.com/rooms/7599950?adults=1&check...,(322),Prix initial :57€Prix réduit :43€ / nuit,Logement entier à 8e arrondissement,4.40,1 voyageur · Studio · 1 lit · Demi-salle de bain
8,Chambre Classique - null - Paris,http://airbnb.com/rooms/35157584?adults=1&chec...,(119),Prix initial :83€Prix réduit :59€ / nuit,Chambre d'hôtel à Tour Eiffel - Champ de Mars,4.73,2 voyageurs · 1 chambre · 1 lit · 1 salle de bain
9,Cute & cosy studio in le Marais Hotel de Ville...,http://airbnb.com/rooms/22155067?adults=1&chec...,(212),Prix initial :137€Prix réduit :79€ / nuit,Logement entier à Le Marais,4.60,2 voyageurs · 1 chambre · 1 lit · 1 salle de bain
10,Chambre Classique Standard - null - Paris,http://airbnb.com/rooms/35149159?adults=1&chec...,(139),Prix initial :71€Prix réduit :51€ / nuit,Chambre dans un boutique-hôtel à 13e arrondiss...,4.71,2 voyageurs · 1 chambre · 1 lit · 1 salle de b...


In [142]:
url_page

'https://www.airbnb.fr/s/Paris/homes?adults=1&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&checkin=2020-11-25&checkout=2020-11-28&source=structured_search_input_header&search_type=filter_change'

In [143]:
## il faut ajouter des dates aux liens pour voir les prix
def findNextPage(soupPage):
	''' Finds the next page with listings if it exists '''
	try:
		nextpage = "https://airbnb.com" + soupPage.find("li", {"class": "_i66xk8d"}).find("a")["href"]
	except: # When he can't find the button, I assume he reached the end
		nextpage = "no next page"
	return nextpage
url=findNextPage(page)
url

'https://airbnb.com/s/Paris/homes?tab_id=home_tab&checkin=2020-11-25&refinement_paths%5B%5D=%2Fhomes&adults=1&source=structured_search_input_header&checkout=2020-11-28&search_type=pagination&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&federated_search_session_id=e7f95f15-9f89-45c7-8fd8-3935b2df26eb&items_offset=20&section_offset=3'

In [144]:
## nombre de page du site web
def getPages(url):
	''' This function returns all the links to the pages containing 
	listings for one particular city '''
	result = []
	while url != "no next page": 
		page = getPage(url)
		result = result + [page]
		url = findNextPage(page)
	return result
URL=getPages(url_page)
len(URL)

14

In [145]:
## Scrapper toutes les infos sur les logements
def extractPages(url):
	''' This function outputs a dataframe that contains all information of a particular
	city. It thus contains information of multiple listings coming from multiple pages.'''
	pages = getPages(url)
	# Do for the first element to initialize the dataframe
	df =extractinfo(pages[0])
	# Loop over all other elements of the dataframe
	for pagenumber in range(1, len(pages)):
		df = df.append(extractinfo(pages[pagenumber]))
	return df
extractPages(url_page)

In [148]:
##base de tous les logements avec les évaluations
data=pd.DataFrame.from_dict(extractPages(url_page))

In [ ]:
## scrapper les commentaires de chaque logement

In [151]:
def setupDriver(url, waiting_time = 2.5):
	''' Initializes the driver of selenium'''
	driver = webdriver.Chrome()
	driver.get(url)
	time.sleep(waiting_time) 
	return driver

In [155]:
def getJSpage(url):
	''' Extracts the html of the webpage including the JS elements,
	output should be used as the input for all functions extracting specific information
	from the detailed pages of the listings '''
	driver = setupDriver(url)
	read_more_buttons = driver.find_elements_by_class_name("_1d079j1e")
	try: # not all pages have buttons
		for i in range(2, len(read_more_buttons)):
			read_more_buttons[i].click()
	except:
		pass
	html = driver.page_source
	driver.close()
	return BeautifulSoup(html, features="lxml") 

getJSpage(url_page)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
